In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import talib
import json
import time
from collections import deque
import warnings
from sklearn.preprocessing import StandardScaler
from pickle import dump, load
from datetime import datetime

file_path = '/content/drive/My Drive/Forex_Robot/'

In [4]:
df_high = pd.read_csv(file_path + 'Oanda_Nzd_Usd_M5.csv')
df_high.reset_index(drop=True, inplace=True)

df_high

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close
0,2020-02-10 08:00:00,0.64042,0.64052,0.64011,0.64031,0.64057,0.64067,0.64028,0.64047,0.64050,0.64060,0.64020,0.64039
1,2020-02-10 08:05:00,0.64030,0.64047,0.64030,0.64041,0.64044,0.64064,0.64044,0.64056,0.64037,0.64056,0.64037,0.64048
2,2020-02-10 08:10:00,0.64039,0.64059,0.64034,0.64058,0.64053,0.64076,0.64049,0.64073,0.64046,0.64068,0.64042,0.64066
3,2020-02-10 08:15:00,0.64061,0.64066,0.64055,0.64059,0.64075,0.64081,0.64071,0.64073,0.64068,0.64073,0.64063,0.64066
4,2020-02-10 08:20:00,0.64061,0.64063,0.64035,0.64042,0.64075,0.64078,0.64050,0.64056,0.64068,0.64070,0.64042,0.64049
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76593,2021-02-19 21:35:00,0.72992,0.73002,0.72992,0.73000,0.73010,0.73020,0.73010,0.73018,0.73001,0.73011,0.73001,0.73009
76594,2021-02-19 21:40:00,0.72998,0.72998,0.72988,0.72989,0.73015,0.73015,0.73005,0.73008,0.73006,0.73006,0.72996,0.72998
76595,2021-02-19 21:45:00,0.72992,0.72992,0.72961,0.72962,0.73010,0.73010,0.72986,0.72990,0.73001,0.73001,0.72974,0.72976
76596,2021-02-19 21:50:00,0.72968,0.72970,0.72951,0.72954,0.72988,0.72991,0.72973,0.72978,0.72978,0.72980,0.72962,0.72966


In [5]:
def add_fractal(df, i, look_back=3):
  if i >= look_back and i < df.shape[0] - look_back:
    lows = []
    highs = []

    for j in range(1, look_back + 1):
      prev_bid_low, prev_bid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
      future_bid_low, future_bid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

      lows.append(float(prev_bid_low))
      lows.append(float(future_bid_low))
      highs.append(float(prev_bid_high))
      highs.append(float(future_bid_high))

    bid_low, bid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

    if float(bid_low) < min(lows):
      return 1

    elif float(bid_high) > max(highs):
      return 2

    else:
      return 0

  else:
    return np.nan

In [6]:
df_high['macd'], df_high['macdsignal'], df_high['macdhist'] = talib.MACD(df_high['Mid_Close'])
df_high['ema200'] = talib.EMA(df_high['Mid_Close'], timeperiod=200)
df_high['slowk'], df_high['slowd'] = talib.STOCH(df_high['Mid_High'], df_high['Mid_Low'], df_high['Mid_Close'])
df_high['fractal'] = [add_fractal(df_high, i) for i in range(df_high.shape[0])]
df_high.dropna(inplace=True)
df_high.reset_index(drop=True, inplace=True)

In [7]:
df_high['fractal'].value_counts()

0.0    63276
1.0     6595
2.0     6525
Name: fractal, dtype: int64

In [9]:
account_balance = 2000
percent_to_risk = 0.05
pips_to_risk = 30
risk_reward_ratio = 2.75
n_units_per_trade = 10000
value_per_pip = n_units_per_trade * 0.0001
pips_to_risk /= 10000
max_pips_to_risk = 0.0100

In [10]:
pips_to_risk = 30 / 10000
risk_reward_ratio = 1.6

In [11]:
df_high.shape

(76396, 20)

In [12]:
def run_simulation(risk_reward_ratio, pullback_cushion, time_window):
  pullback_cushion /= 10000
  reward = 0
  n_wins = 0
  n_losses = 0
  win_streak = 0
  loss_streak = 0
  curr_win_streak = 0
  curr_loss_streak = 0
  n_buys = 0
  n_sells = 0
  length_divider = 24 * 12
  pips_risked = []
  day_fees = 0
  stoch_signal = None
  z = 0
  n_units = 50000
  trade = None

  for i in range(2, df_high.shape[0] - 1):
    if z >= time_window:
      stoch_signal = None
      z = 0
    
    slowk1 = df_high.loc[df_high.index[i - 1], 'slowk']
    slowd1 = df_high.loc[df_high.index[i - 1], 'slowd']
    slowk2 = df_high.loc[df_high.index[i - 2], 'slowk']
    slowd2 = df_high.loc[df_high.index[i - 2], 'slowd']
    macd1 = df_high.loc[df_high.index[i - 1], 'macd']
    macdsignal1 = df_high.loc[df_high.index[i - 1], 'macdsignal']
    macd2 = df_high.loc[df_high.index[i - 2], 'macd']
    macdsignal2 = df_high.loc[df_high.index[i - 2], 'macdsignal']
    ema200 = df_high.loc[df_high.index[i - 1], 'ema200']
    bid_low1 = df_high.loc[df_high.index[i - 1], 'Mid_Low']
    bid_high1 = df_high.loc[df_high.index[i - 1], 'Mid_High']
    bid_open1 = df_high.loc[df_high.index[i - 1], 'Mid_Open']
    bid_close1 = df_high.loc[df_high.index[i - 1], 'Mid_Close']

    if slowk2 < slowd2 and slowk1 > slowd1 and max([20, slowk2, slowd2, slowk1, slowd1]) == 20:
      stoch_signal = 'buy'
      z = 0

    elif slowk2 > slowd2 and slowk1 < slowd1 and min([80, slowk2, slowd2, slowk1, slowd1]) == 80:
      stoch_signal = 'sell'
      z = 0

    if macd2 < macdsignal2 and macd1 > macdsignal1 and bid_low1 > ema200 and stoch_signal == 'buy' and trade is None:
      pullback = None
      j = i - 4

      while j >= 0:
        curr_fractal = df_high.loc[df_high.index[j], 'fractal']

        if curr_fractal == 1:
          pullback = float(df_high.loc[df_high.index[j], 'Ask_Low'])
          break

        j -= 1

      if pullback is not None:
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = df_high.loc[df_high.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close']]
        open_price = float(curr_ask_open)
        stop_loss = round(pullback - pullback_cushion, 5)

        if stop_loss < open_price:
          curr_pips_to_risk = open_price - stop_loss

          if curr_pips_to_risk <= max_pips_to_risk:
            stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)

            trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss, 'stop_gain': stop_gain, 'length': 0, 'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units}

            n_buys += 1

            pips_risked.append(curr_pips_to_risk)

            stoch_signal = None

    elif macd2 > macdsignal2 and macd1 < macdsignal1 and bid_high1 < ema200 and stoch_signal == 'sell' and trade is None:
      pullback = None
      j = i - 4

      while j >= 0:
        curr_fractal = df_high.loc[df_high.index[j], 'fractal']

        if curr_fractal == 2:
          pullback = float(df_high.loc[df_high.index[j], 'Bid_High'])
          break

        j -= 1

      if pullback is not None:
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = df_high.loc[df_high.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close']]
        open_price = float(curr_bid_open)
        stop_loss = round(pullback + pullback_cushion, 5)

        if stop_loss > open_price:
          curr_pips_to_risk = stop_loss - open_price

          if curr_pips_to_risk <= max_pips_to_risk:
            stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)

            trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss, 'stop_gain': stop_gain, 'length': 0, 'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units}

            n_sells += 1

            pips_risked.append(curr_pips_to_risk)

            stoch_signal = None

    if stoch_signal is not None:
      z += 1

    if trade is not None:
      trade['length'] += 1

      curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = df_high.loc[df_high.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Ask_Close']]
      next_bid_open, next_ask_open = df_high.loc[df_high.index[i + 1], ['Bid_Open', 'Ask_Open']]

      if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
          trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees = day_fees + (-0.8 * int(trade['length'] / length_divider))

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0

          trade = None

          continue

      if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
          trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees = day_fees + (-0.8 * int(trade['length'] / length_divider))

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0

          trade = None

          continue

      if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
          trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees = day_fees + (-0.8 * int(trade['length'] / length_divider))

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0

          trade = None

          continue

      if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
          trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees = day_fees + (-0.8 * int(trade['length'] / length_divider))

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0

          trade = None

          continue

  return reward + day_fees, n_buys + n_sells

In [14]:
risk_reward_ratio_vals = [1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2]
pullback_cushion_vals = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
time_windows = [6, 7, 8, 9, 10]
n_runs = len(risk_reward_ratio_vals) * len(pullback_cushion_vals) * len(time_windows)
best_risk_reward = None
best_pullback_cushion = None
best_time_window = None
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio in risk_reward_ratio_vals:
  for pullback_cushion in pullback_cushion_vals:
    for time_window in time_windows:
      reward, n_trades = run_simulation(risk_reward_ratio, pullback_cushion, time_window)
      runs_finished += 1

      print(risk_reward_ratio)
      print(pullback_cushion)
      print(time_window)
      print(reward)
      print('Num trades: ' + str(n_trades))
      print('Remaining runs: ' + str(n_runs - runs_finished))

      if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_pullback_cushion = pullback_cushion
        best_time_window = time_window

      print('Best reward so far: ' + str(best_reward))
      print()

print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best pullback cushion: ' + str(best_pullback_cushion))
print('Best time_window: ' + str(best_time_window))

1.2
5
6
-1183.599999999966
Num trades: 157
Remaining runs: 449
Best reward so far: -1183.599999999966

1.2
5
7
-3511.89999999996
Num trades: 224
Remaining runs: 448
Best reward so far: -1183.599999999966

1.2
5
8
-4244.69999999996
Num trades: 274
Remaining runs: 447
Best reward so far: -1183.599999999966

1.2
5
9
-4368.200000000003
Num trades: 319
Remaining runs: 446
Best reward so far: -1183.599999999966

1.2
5
10
-4681.399999999992
Num trades: 362
Remaining runs: 445
Best reward so far: -1183.599999999966

1.2
10
6
-21.099999999991347
Num trades: 137
Remaining runs: 444
Best reward so far: -21.099999999991347

1.2
10
7
-1741.4000000000028
Num trades: 195
Remaining runs: 443
Best reward so far: -21.099999999991347

1.2
10
8
-2563.500000000026
Num trades: 238
Remaining runs: 442
Best reward so far: -21.099999999991347

1.2
10
9
-2203.100000000037
Num trades: 273
Remaining runs: 441
Best reward so far: -21.099999999991347

1.2
10
10
-2484.3000000000243
Num trades: 305
Remaining runs: 44

KeyboardInterrupt: ignored

In [15]:
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best pullback cushion: ' + str(best_pullback_cushion))
print('Best time_window: ' + str(best_time_window))

Best reward: 3231.000000000024
Best risk/reward ratio: 1.6
Best pullback cushion: 50
Best time_window: 6
